# Limpieza de datos y series de tiempo

In [ ]:
# Librerías necesarias
import pandas as pd
import glob
import os
import unicodedata

Carga de datos

In [2]:
# Definimos la ruta de la carpeta que contiene los archivos CSV
folder_path = '../data/'

# Encontramos todos los archivos CSV en la carpeta especificada
all_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Leemos y concatenamos todos los archivos CSV en una lista
li = []

for filename in all_files:

    df = pd.read_csv(filename, header=4, encoding='latin-1')
    li.append(df)

# Concatenamos todos los DataFrames
final_df = pd.concat(li, axis=0, ignore_index=True)

# Mostramos la información del DataFrame final
print(f"Archivos CSV encontrados y concatenados: {len(li)}")

Archivos CSV encontrados y concatenados: 23


In [6]:
final_df.head(5)

,Año,Mes,Fecha_Pub_DOF,Clave ciudad,Nombre ciudad,División,Grupo,Clase,Subclase,Clave genérico,Genérico,Consecutivo,Especificación,Precio promedio,Cantidad,Unidad,Estatus
0,2011,4,26/05/2011 12:00:00 a. m.,1,Área Met. de la Cd. de México,"1. Alimentos, bebidas y tabaco",1.1. Alimentos,"1.1.1. Pan, tortillas y cereales",01 Tortillas y derivados del maíz,1,Tortilla de maíz,1,"TORTILLAS, A GRANEL",9.88,1,KG,NaN
1,2011,4,26/05/2011 12:00:00 a. m.,1,Área Met. de la Cd. de México,"1. Alimentos, bebidas y tabaco",1.1. Alimentos,"1.1.1. Pan, tortillas y cereales",01 Tortillas y derivados del maíz,1,Tortilla de maíz,2,"TORTILLAS, A GRANEL",7.58,1,KG,NaN
2,2011,4,26/05/2011 12:00:00 a. m.,1,Área Met. de la Cd. de México,"1. Alimentos, bebidas y tabaco",1.1. Alimentos,"1.1.1. Pan, tortillas y cereales",01 Tortillas y derivados del maíz,1,Tortilla de maíz,3,"TORTILLAS, A GRANEL",10.00,1,KG,NaN
3,2011,4,26/05/2011 12:00:00 a. m.,1,Área Met. de la Cd. de México,"1. Alimentos, bebidas y tabaco",1.1. Alimentos,"1.1.1. Pan, tortillas y cereales",01 Tortillas y derivados del maíz,1,Tortilla de maíz,4,"TORTILLAS, A GRANEL",10.00,1,KG,NaN
4,2011,4,26/05/2011 12:00:00 a. m.,1,Área Met. de la Cd. de México,"1. Alimentos, bebidas y tabaco",1.1. Alimentos,"1.1.1. Pan, tortillas y cereales",01 Tortillas y derivados del maíz,1,Tortilla de maíz,5,"TORTILLAS, A GRANEL",11.00,1,KG,NaN


In [ ]:
def limpiar_nombre(texto):
    """
    Función auxiliar para limpiar strings:
    1. Quita acentos
    2. Quita comas y puntos
    3. Pasa a minúsculas
    4. Reemplaza espacios por guiones bajos
    """
    # Normalizar unicode: separa la 'á' en 'a' + '´'
    texto_nfkd = unicodedata.normalize('NFKD', str(texto))
    
    # Codificar a ASCII ignorando errores (esto elimina las tildes separadas)
    texto_sin_tildes = texto_nfkd.encode('ASCII', 'ignore').decode('utf-8')
    
    # Quitar comas y puntos (puedes agregar más caracteres si quieres)
    texto_limpio = texto_sin_tildes.replace(',', '').replace('.', '')
    
    # Formato final: minúsculas y espacios a guiones bajos
    return texto_limpio.strip().replace(' ', '_').lower()

In [ ]:
def comida_datasets(df):
    """
    Función para crear datasets de series de tiempo para cada alimento.
    Parámetros:
    df (DataFrame): DataFrame con los datos originales.
    Retorna: None
    Guarda archivos CSV en la carpeta de preferencia.
    """
    # Obtenemos los alimentos únicos en la columna 'Genérico'
    alimentos_unicos = df['Genérico'].unique()
    df = df[['Año', 'Mes', 'Genérico', 'Precio promedio']].copy()
    df['tiempo_str'] = df['Año'].astype(str) + '-' + df['Mes'].astype(str).str.zfill(2) + '-01'

    # Convertimos la nueva columna a formato datetime
    df['Tiempo'] = pd.to_datetime(df['tiempo_str'])
    # eliminamos columnas auxiliares
    df = df.drop(columns=['tiempo_str', 'Año', 'Mes'])
    # convertimos Time a indice
    df = df.set_index('Tiempo')
    # Realizamos el preprocesamiento para cada alimento
    for alimento in alimentos_unicos:
        # nos fijamos en precio de tortillas de maiz 
        df_filter = df[df['Genérico'] == alimento].copy()
        # Nos quedamos solo con la columna de precio promedio
        df_filter = df_filter[['Precio promedio']]
        # agrupamos por mes y año 
        df_filter_mensual = df_filter.resample('ME').mean()
        # limpiamos nombre 
        nombre_archivo = limpiar_nombre(alimento)
        # guardamos el csv
        df_filter_mensual.to_csv(f'../data/precios_comida/{nombre_archivo}.csv')
        print(f"Guardado: {nombre_archivo}.csv")
    
    return None

In [240]:
comida_datasets(final_df)

Guardado: tortilla_de_maiz.csv
Guardado: tostadas.csv
Guardado: masa_y_harinas_de_maiz.csv
Guardado: maiz.csv
Guardado: pan_dulce.csv
Guardado: pan_blanco.csv
Guardado: pan_de_caja.csv
Guardado: pasteles_pastelillos_y_pan_dulce_empaquetado.csv
Guardado: pastelillos_y_pasteles_a_granel.csv
Guardado: galletas.csv
Guardado: pasta_para_sopa.csv
Guardado: tortillas_de_harina_de_trigo.csv
Guardado: harinas_de_trigo.csv
Guardado: cereales_en_hojuelas.csv
Guardado: arroz.csv
Guardado: pollo.csv
Guardado: carne_de_cerdo.csv
Guardado: carne_de_res.csv
Guardado: visceras_de_res.csv
Guardado: chorizo.csv
Guardado: jamon.csv
Guardado: salchichas.csv
Guardado: carnes_secas_y_otros_embutidos.csv
Guardado: tocino.csv
Guardado: pescado.csv
Guardado: camaron.csv
Guardado: otros_mariscos.csv
Guardado: atun_y_sardina_en_lata.csv
Guardado: leche_pasteurizada_y_fresca.csv
Guardado: leche_en_polvo.csv
Guardado: leche_evaporada_condensada_y_maternizada.csv
Guardado: yogurt.csv
Guardado: queso_fresco.csv
Guard

In [ ]:
# Limpiamos los nombres de los alimentos y creamos un diccionario
alimentos = final_df['Genérico'].unique()
unidad_dict = {}
for alimento in alimentos:
    nombre_limpio = limpiar_nombre(alimento)
    unidad = final_df[final_df['Genérico'] == alimento]['Unidad'].iloc[0]
    unidad_dict[nombre_limpio] = unidad
print(unidad_dict)

{'tortilla_de_maiz': 'KG', 'tostadas': 'KG', 'masa_y_harinas_de_maiz': 'KG', 'maiz': 'KG', 'pan_dulce': 'PZA', 'pan_blanco': 'PZA', 'pan_de_caja': 'KG', 'pasteles_pastelillos_y_pan_dulce_empaquetado': 'KG', 'pastelillos_y_pasteles_a_granel': 'KG', 'galletas': 'KG', 'pasta_para_sopa': 'KG', 'tortillas_de_harina_de_trigo': 'KG', 'harinas_de_trigo': 'KG', 'cereales_en_hojuelas': 'KG', 'arroz': 'KG', 'pollo': 'KG', 'carne_de_cerdo': 'KG', 'carne_de_res': 'KG', 'visceras_de_res': 'KG', 'chorizo': 'KG', 'jamon': 'KG', 'salchichas': 'KG', 'carnes_secas_y_otros_embutidos': 'KG', 'tocino': 'KG', 'pescado': 'KG', 'camaron': 'KG', 'otros_mariscos': 'KG', 'atun_y_sardina_en_lata': 'KG', 'leche_pasteurizada_y_fresca': 'LT', 'leche_en_polvo': 'KG', 'leche_evaporada_condensada_y_maternizada': 'KG', 'yogurt': 'KG', 'queso_fresco': 'KG', 'otros_quesos': 'KG', 'queso_oaxaca_o_asadero': 'KG', 'crema_de_leche': 'KG', 'queso_manchego_o_chihuahua': 'KG', 'helados': 'LT', 'mantequilla': 'KG', 'queso_amarillo

In [243]:
# guardar el diccionario de unidades en un archivo JSON
import json 
with open('../data/unidad_dict.json', 'w') as f:
    json.dump(unidad_dict, f)

# Predicción de series temporales para alimentos específicos

In [46]:
import pandas as pd
from prophet import Prophet
import joblib
import os

class FoodPricePipeline:
    def __init__(self, model_dir='modelos_alimentos_ale'): # cambiar ruta a carpeta de modelos
        """
        Inicializa el pipeline.
        model_dir: Carpeta donde se guardarán los modelos entrenados.
        """
        self.model_dir = model_dir
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        self.models = {}

    def preprocess_data(self, raw_data):
        """
        Limpia y prepara los datos para Prophet.
        Espera un DataFrame con columnas 'Tiempo' y 'Precio promedio'.
        """
        df = raw_data.copy()
        df['ds'] = pd.to_datetime(df['Tiempo'])
        df['y'] = df['Precio promedio']
        return df[['ds', 'y']]

    def train(self, food_name, raw_data):
        """
        Entrena el modelo para un alimento específico.
        """
        print(f" entrenando modelo para: {food_name}...")
        df = self.preprocess_data(raw_data)
        
        # Configuración para largo plazo:
        # changepoint_prior_scale: flexibilidad ante cambios de tendencia
        # yearly_seasonality: Vital para alimentos (ciclos de cosecha)
        model = Prophet(
            yearly_seasonality=False, 
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=0.05 
        )
        
        model.fit(df)
        
        # Guardamos el modelo en memoria y disco
        self.models[food_name] = model
        joblib.dump(model, os.path.join(self.model_dir, f'{food_name}_model.pkl'))
        print(f" Modelo para {food_name} guardado exitosamente.")

    def predict(self, food_name, months_ahead=24): # 96 para primeros modelos
        """
        Genera la predicción a futuro.
        months_ahead: 96 meses es la distancia aprox de 2018 a 2026.
        """
        # Cargar modelo si no está en memoria
        if food_name not in self.models:
            try:
                path = os.path.join(self.model_dir, f'{food_name}_model.pkl')
                self.models[food_name] = joblib.load(path)
            except FileNotFoundError:
                return f"Error: No existe un modelo entrenado para {food_name}"

        model = self.models[food_name]
        
        # Crear el dataframe futuro
        future = model.make_future_dataframe(periods=months_ahead, freq='M')
        
        # Predecir
        forecast = model.predict(future)
        
        # Retornar solo las columnas relevantes y los últimos datos (2026)
        result = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(12) # Último año
        return result

Creación de modelos con base a datos históricos de precios de alimentos para predecir tendencias futuras.

In [47]:
ruta_alimentos = 'D:\\7mo_Semestre\\machine\\final_project\\datos_ale\\Inegi_actualizados\\Inegi_actualizados\\'

# Encontramos todos los archivos CSV en la carpeta especificada
all_files = glob.glob(os.path.join(ruta_alimentos, "*.csv"))

# Leemos y concatenamos todos los archivos CSV en una lista
li = []

for filename in all_files:

    df_user = pd.read_csv(filename)
    pipeline = FoodPricePipeline()
    alimento_nombre = os.path.basename(filename).replace('.csv', '')
    pipeline.train(alimento_nombre, df_user)

10:36:24 - cmdstanpy - INFO - Chain [1] start processing
10:36:24 - cmdstanpy - INFO - Chain [1] done processing


 entrenando modelo para: aceites_y_grasas_vegetales_comestibles...
 Modelo para aceites_y_grasas_vegetales_comestibles guardado exitosamente.
 entrenando modelo para: aguacate...


10:36:25 - cmdstanpy - INFO - Chain [1] start processing
10:36:25 - cmdstanpy - INFO - Chain [1] done processing
10:36:25 - cmdstanpy - INFO - Chain [1] start processing
10:36:25 - cmdstanpy - INFO - Chain [1] done processing


 Modelo para aguacate guardado exitosamente.
 entrenando modelo para: arroz...
 Modelo para arroz guardado exitosamente.
 entrenando modelo para: azucar...


10:36:25 - cmdstanpy - INFO - Chain [1] start processing
10:36:25 - cmdstanpy - INFO - Chain [1] done processing
10:36:25 - cmdstanpy - INFO - Chain [1] start processing
10:36:25 - cmdstanpy - INFO - Chain [1] done processing


...

 Modelo para tomate_verde guardado exitosamente.
 entrenando modelo para: tortilla_de_maiz...


10:36:31 - cmdstanpy - INFO - Chain [1] done processing
10:36:31 - cmdstanpy - INFO - Chain [1] start processing
10:36:31 - cmdstanpy - INFO - Chain [1] done processing


 Modelo para tortilla_de_maiz guardado exitosamente.
 entrenando modelo para: uva...
 Modelo para uva guardado exitosamente.
 entrenando modelo para: zanahoria...


10:36:31 - cmdstanpy - INFO - Chain [1] start processing
10:36:31 - cmdstanpy - INFO - Chain [1] done processing


 Modelo para zanahoria guardado exitosamente.


# Verificacion 

In [ ]:
import joblib
import pandas as pd
import os
import unicodedata

class ConsultorDePrecios:
    def __init__(self, carpetas_modelos='modelos_alimentos_ale'):
        self.carpetas_modelos = carpetas_modelos

    def _limpiar_nombre(self, texto):
        """
        Reutilizamos la misma lógica de limpieza para que si el usuario
        escribe "Maíz", el sistema encuentre 'maiz.pkl'
        """
        texto_nfkd = unicodedata.normalize('NFKD', str(texto))
        texto_sin_tildes = texto_nfkd.encode('ASCII', 'ignore').decode('utf-8')
        texto_limpio = texto_sin_tildes.replace(',', '').replace('.', '')
        return texto_limpio.strip().replace(' ', '_').lower()

    def obtener_prediccion(self, nombre_alimento, fecha_objetivo='2026-07-01'):
        """
        Carga el modelo y devuelve la predicción para la fecha exacta.
        """
        # Limpiamos el nombre para encontrar el archivo
        nombre_archivo = self._limpiar_nombre(nombre_alimento)
        ruta_modelo = os.path.join(self.carpetas_modelos, f'{nombre_archivo}_model.pkl')

        # Verificamos si existe el modelo
        if not os.path.exists(ruta_modelo):
            return f"Error: No tengo un modelo entrenado para '{nombre_alimento}' (busqué '{nombre_archivo}_model.pkl')"

        # Cargamos el modelo (Deserialización)
        modelo = joblib.load(ruta_modelo)

        # Calculamos cuántos meses faltan desde el FINAL de los datos de entrenamiento
        # Prophet necesita saber el último punto de historia para proyectar
        # Nota: Accedemos al historial interno de Prophet para ver su última fecha
        ultima_fecha_entrenamiento = modelo.history['ds'].max()
        fecha_obj_dt = pd.to_datetime(fecha_objetivo)
        
        # Calculamos la diferencia en meses aproximada
        diferencia = (fecha_obj_dt.year - ultima_fecha_entrenamiento.year) * 12 + \
                     (fecha_obj_dt.month - ultima_fecha_entrenamiento.month)
        
        if diferencia < 1:
            return "La fecha solicitada es anterior o igual al último dato histórico."

        # Creamos el dataframe futuro
        # Periods = meses a futuro + un pequeño margen de seguridad
        future = modelo.make_future_dataframe(periods=diferencia + 1, freq='M')
        
        # Predecimos
        forecast = modelo.predict(future)

        # Extraemos solo el dato de la fecha que queremos
        # Filtramos la fila más cercana a la fecha objetivo
        prediccion_fila = forecast.iloc[-1] # Tomamos la última que calculamos
        
        precio_estimado = round(prediccion_fila['yhat'], 2)
        minimo_estimado = round(prediccion_fila['yhat_lower'], 2)
        maximo_estimado = round(prediccion_fila['yhat_upper'], 2)

        return {
            "alimento": nombre_alimento,
            "fecha_prediccion": fecha_objetivo,
            "precio_esperado": precio_estimado,
            "precio_min" : minimo_estimado,
            "precio_max" : maximo_estimado
        }


--- RESULTADO ---
{'alimento': 'Tortilla de maíz', 'fecha_prediccion': '2026-02-01', 'precio_esperado': 16.22, 'precio_min': 15.93, 'precio_max': 16.5}


d:\7mo_Semestre\7mo\Lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [107]:
# --- EJEMPLO DE USO ---

# 1. Iniciamos el consultor
consultor = ConsultorDePrecios(carpetas_modelos='D:\\7mo_Semestre\\machine\\final_project\\src\\modelos_alimentos_ale') # Ajusta tu ruta
#consultor = ConsultorDePrecios(carpetas_modelos='../modelos_alimentos/') # poner ruta de modelos
# 2. El usuario pregunta
resultado = consultor.obtener_prediccion("limon", fecha_objetivo='2026-12-01')

print("\n--- RESULTADO ---")
print(resultado)


--- RESULTADO ---
{'alimento': 'limon', 'fecha_prediccion': '2026-12-01', 'precio_esperado': 31.05, 'precio_min': 21.5, 'precio_max': 40.66}


d:\7mo_Semestre\7mo\Lib\site-packages\prophet\forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
